In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!mkdir input_data
!unzip "/content/gdrive/My Drive/Bases_VisagioHackathon.zip" -d "/content/input_data"


Mounted at /content/gdrive
Archive:  /content/gdrive/My Drive/Bases_VisagioHackathon.zip
  inflating: /content/input_data/Base_tweets_saúde_Aos_Fatos.xlsx  
   creating: /content/input_data/FakeHealthData/
  inflating: /content/input_data/FakeHealthData/Base News Releases.csv  
  inflating: /content/input_data/FakeHealthData/Base Story.csv  
  inflating: /content/input_data/FakeHealthData/de_para_news_questions.csv  
  inflating: /content/input_data/FakeHealthData/de_para_story_questions.csv  
  inflating: /content/input_data/FakeHealthData/news_tweets.csv  
  inflating: /content/input_data/FakeHealthData/story_tweets.csv  


In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!git pull
!pip install --upgrade .
!pip install sentencepiece
!pip install transformers
import torch
import sentencepiece
import gc
gc.collect()

Cloning into 'transformers'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 54705 (delta 20), reused 21 (delta 8), pack-reused 54656
Receiving objects: 100% (54705/54705), 40.82 MiB | 32.40 MiB/s, done.
Resolving deltas: 100% (38251/38251), done.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.9MB 8.6MB/s 
     |████████████████████████████████| 890kB 58.9MB/s 
  Created wheel for transformers: filename=transformers-4.1.0.dev0-cp36-none-any.whl size=1375102 sha256=0fc025bc57184a88eff681225b9a47f600f899597bd6551fd7459324ced350d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-htnsbtt2/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Created wheel for sacremoses

20

In [ ]:
import pandas as pd
base_news_releases = pd.read_csv('/content/input_data/FakeHealthData/Base News Releases.csv',encoding='utf-8',sep=';')
base_news_releases.head(2)

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title_1,meta_data,movies,publish_date,source,summary,id,link,title_2,description,original_title,rating,reviewers,category,tags,source_link,news_id,news_source,Our Review Summary,Why This Matters,img,question_0_answer,question_0_explanation,question_1_answer,question_1_explanation,question_2_answer,question_2_explanation,question_3_answer,question_3_explanation,question_4_answer,question_4_explanation,question_5_answer,question_5_explanation,question_6_answer,question_6_explanation,question_7_answer,question_7_explanation,question_8_answer,question_8_explanation,question_9_answer,question_9_explanation
0,0,https://web.archive.org/web/20181218015531/htt...,"MADISON, Wis. -- More than 700 million adults ...",['https://web.archive.org/web/20181218015531im...,https://web.archive.org/web/20181218015531im_/...,[],[],https://web.archive.org/web/20181218015531/htt...,Tiny implantable device short-circuits hunger ...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00000,https://www.healthnewsreview.org/news-release-...,Implantable device ‘aids weight loss’? Let’s w...,This news release speculates that a study in r...,Tiny implantable device short-circuits hunger ...,2,"['Andrew Holtz, MPH', 'Yoni Freedhoff, MD', 'K...",University news release,"['implant', 'University of Wisconsin-Madison',...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00000,University of Wisconsin-Madison,UW-Madison graduate student Guang Yao (left) a...,The history of medical research is littered wi...,NaN,Not Satisfactory,Although this device is still in early testing...,Satisfactory,The release states that the device is experime...,Satisfactory,The release does report that there is an exist...,Not Satisfactory,"The release does not discuss potential harms, ...",Satisfactory,We will give the release a satisfactory rating...,Not Satisfactory,"By definition, researchers do not know whether...",Not Satisfactory,The lead sentence of the release proclaims tha...,Not Satisfactory,The news release refers to “several advantages...,Satisfactory,The release states that funding was provided b...,Not Satisfactory,The release says this device could be an impro...
1,1,https://web.archive.org/web/20181217203805/htt...,"Wilmington, DE, December 17, 2018 - The CRISPR...",['https://web.archive.org/web/20181217203805im...,https://web.archive.org/web/20181217203805im_/...,[],[],https://web.archive.org/web/20181217203805/htt...,Scientists report CRISPR restores effectivenes...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00001,https://www.healthnewsreview.org/news-release-...,Summary of CRISPR lung cancer research downpla...,This release needed a strong caution that obse...,Scientists report CRISPR restores effectivenes...,3,"['Mary Chris Jaklevic', 'Andrae Vandross, MD',...",Academic medical center news release,"['Christiana Care Health System', 'CRISPR', 'l...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00001,Christiana Care Health System,This news release reports on preliminary resea...,Chemotherapy is a mainstay of treatment for ad...,https://www.healthnewsreview.org/wp-content/up...,Not Satisfactory,The news release mentioned this as a potential...,Satisfactory,The news release makes it clear that this is n...,Satisfactory,The news release mentions “many efforts under ...,Not Satisfactory,The news release doesn’t talk about specific h...,Not Satisfactory,In the second paragraph the news release expla...,Not Satisfactory,The news release uses cautious language. Howev...,Satisfactory,There’s no disease-mongering. The release says...,Not Satisfactory,The release is unable to claim a benefit in hu...,Satisfactory,The funders (NIH and State of Delaware) are no...,Satisfactory,The lead researcher calls the study “an exciti...


In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [ ]:
def summary_generate(text,model,tokenizer):
  '''
    Generates the summary of a text based on a given model.
  '''

  if len(text.split())>4200:
    max_length_in = 600
    min_length_in = 300
  
  elif ((len(text.split())<4200) and (len(text.split())>3000)):
    max_length_in = 500
    min_length_in = 200

  else:
    max_length_in = 200
    min_length_in = 80
  
  batch        = tokenizer.prepare_seq2seq_batch([text], truncation=True, padding='longest', return_tensors="pt").to(torch_device)
  translated   = model.generate(
                                **batch,
                                max_length = 500, # max length of summary
                                min_length = 200, # min length of summary
                                do_sample = True, 
                                temperature = 3.0,
                                top_k =30,
                                top_p=0.70,
                                repetition_penalty = 1.2,
                                length_penalty = 5, # if more than 1 encourage model to generate #larger sequences
                                num_return_sequences=1) # no of summary you want to generate)
  tgt_text     = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text[0]


In [ ]:
model_name   = 'google/pegasus-xsum'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer    = PegasusTokenizer.from_pretrained(model_name)
model        = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

In [ ]:
base_news_releases['text'] = base_news_releases['text'].astype(str)

In [ ]:
summary_result = []
for i in range(base_news_releases.shape[0]):
  
  text = base_news_releases.loc[i,'text']
  summary = summary_generate(text,model,tokenizer)
  summary_result.append(summary)
  print('Summary Generated:-{}'.format(summary))
  print('\n')

Summary Generated:-Researchers have developed an electric stimulator that boosts hunger-control in the stomach, allowing them to shed the extra pounds and get back to being the weight they were before obesity began to take a toll at an alarming rate around the world: 70 percent of men and women now have a waist-line of 25 or more inches, while nearly a quarter of US children aged 1 to 8 are overweight, with nearly a fifth having a BMI over 36", which represents an extra 14 pounds of fat over a teenager's weight-control threshold, and 10 to 11 pounds above a young adult's weight - a "worrying epidemic" according to one prominent health expert, and one that could have long-term consequences on mental health and social development if it weren't for effective and reversible weight-loss treatments like surgical bariatric operations, which involve permanently altering a patient's stomach to remove some of its excess capacity to hold food, but are currently used for just about every other typ

In [ ]:

!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(t2,
                      text)
scores 

{'rouge1': Score(precision=0.1367965367965368, recall=0.6929824561403509, fmeasure=0.2284887924801157),
 'rougeL': Score(precision=0.05800865800865801, recall=0.29385964912280704, fmeasure=0.09689081706435286)}

In [ ]:
from rouge_score import rouge_scorer



In [ ]:
!pip install rouge_score

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)


output = []
for i in range(len(summary_result)):
  t1 = summary_result[i]
  t2 = base_news_releases.loc[i,'Our Review Summary']
  text = base_news_releases.loc[i,'text']
  rating = base_news_releases.loc[i,'rating']
  id     = base_news_releases.loc[i,'id']
  
  scores_orignal_summary   = scorer.score(t2,text)
  fmeasure_orig_1          = scores_orignal_summary['rouge1'][2]
  fmeasure_orig_L          = scores_orignal_summary['rougeL'][2]
  
  scores_generated_summary = scorer.score(t1,text)
  fmeasure_gen_1           = scores_generated_summary['rouge1'][2]
  fmeasure_gen_L           = scores_generated_summary['rougeL'][2]
  output.append([id,text,t1,t2,fmeasure_gen_1,fmeasure_gen_L,fmeasure_orig_1,fmeasure_orig_L,rating])

In [ ]:
df = pd.DataFrame(output,columns=['ID','Text','generated_summary','Original_summary',
                           'Rouge_score_Uni_gen','Rouge_score_fulllength_gen',
                           'Rouge_score_Uni_orig','Rouge_score_fulllength_orig','rating'
                           ])
df['ID'] =list(range(1,301))
df.to_csv('/content/output3.csv',index=False)

In [ ]:
df.tail()

,ID,Text,generated_summary,Original_summary,Rouge_score_Uni_gen,Rouge_score_fulllength_gen,Rouge_score_Uni_orig,Rouge_score_fulllength_orig,rating
295,296,'There are women who are alive today because o...,The first trial in breast cancer to show promi...,This news release from the University of Color...,0.269841,0.126984,0.234339,0.127610,1
296,297,A new pilot study led by McLean Hospital's Ips...,Tablets have quickly become the dominant elect...,This news release describes a feasibility stud...,0.237500,0.110000,0.175637,0.099150,4
297,298,A new method of stimulating the renewal of liv...,Researchers believe that the natural repairs m...,One of the most basic elements of any news rel...,0.251343,0.120301,0.213421,0.129813,0
298,299,- Shows Efficacy at 12 Weeks of Treatment\n\n-...,"Sun Pharmaceutical Industries Limited, a globa...",This release reports on the results of a phase...,0.178423,0.089212,0.159116,0.092818,2
299,300,"Oakland, CA (January 3, 2017) - A new study by...",The humble rice and wheat have been known for ...,This news release describes a new study lookin...,0.228869,0.109233,0.269861,0.136192,2


In [ ]:
text

'Wilmington, DE, December 17, 2018 - The CRISPR-Cas9 gene editing system may be able to restore the effectiveness of first-line chemotherapies used to treat lung cancer by deleting or "knocking out" a gene in cancer tumors that helps the tumors develop resistance to the drugs. That was the conclusion of a new study published today in the journal Molecular Therapy Oncolytics by scientists from The Gene Editing Institute of the Helen F. Graham Cancer Center & Research Institute at Christiana Care Health System.\n\nThe study reports that in both tissue culture and in a mouse, tumor growth stopped and there was a dramatic decrease in the volume of existing tumors when chemotherapy was combined with CRIPSP-Cas9, which was used to disable a tumor gene known as NRF2. Previous studies have shown that the NRF2 gene controls cell functions in lung cancer tumors that helps them thwart the effect of chemotherapies that might otherwise reduce or eliminate them entirely.\n\n"Our goal is to see if CR

In [ ]:
base_news_releases['length']=base_news_releases['text'].str.len()

In [ ]:
base_news_releases.head()

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title_1,meta_data,movies,publish_date,source,summary,id,link,title_2,description,original_title,rating,reviewers,category,tags,source_link,news_id,news_source,Our Review Summary,Why This Matters,img,question_0_answer,question_0_explanation,question_1_answer,question_1_explanation,question_2_answer,question_2_explanation,question_3_answer,question_3_explanation,question_4_answer,question_4_explanation,question_5_answer,question_5_explanation,question_6_answer,question_6_explanation,question_7_answer,question_7_explanation,question_8_answer,question_8_explanation,question_9_answer,question_9_explanation
0,0,https://web.archive.org/web/20181218015531/htt...,"MADISON, Wis. -- More than 700 million adults ...",['https://web.archive.org/web/20181218015531im...,https://web.archive.org/web/20181218015531im_/...,[],[],https://web.archive.org/web/20181218015531/htt...,Tiny implantable device short-circuits hunger ...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00000,https://www.healthnewsreview.org/news-release-...,Implantable device ‘aids weight loss’? Let’s w...,This news release speculates that a study in r...,Tiny implantable device short-circuits hunger ...,2,"['Andrew Holtz, MPH', 'Yoni Freedhoff, MD', 'K...",University news release,"['implant', 'University of Wisconsin-Madison',...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00000,University of Wisconsin-Madison,UW-Madison graduate student Guang Yao (left) a...,The history of medical research is littered wi...,NaN,Not Satisfactory,Although this device is still in early testing...,Satisfactory,The release states that the device is experime...,Satisfactory,The release does report that there is an exist...,Not Satisfactory,"The release does not discuss potential harms, ...",Satisfactory,We will give the release a satisfactory rating...,Not Satisfactory,"By definition, researchers do not know whether...",Not Satisfactory,The lead sentence of the release proclaims tha...,Not Satisfactory,The news release refers to “several advantages...,Satisfactory,The release states that funding was provided b...,Not Satisfactory,The release says this device could be an impro...
1,1,https://web.archive.org/web/20181217203805/htt...,"Wilmington, DE, December 17, 2018 - The CRISPR...",['https://web.archive.org/web/20181217203805im...,https://web.archive.org/web/20181217203805im_/...,[],[],https://web.archive.org/web/20181217203805/htt...,Scientists report CRISPR restores effectivenes...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00001,https://www.healthnewsreview.org/news-release-...,Summary of CRISPR lung cancer research downpla...,This release needed a strong caution that obse...,Scientists report CRISPR restores effectivenes...,3,"['Mary Chris Jaklevic', 'Andrae Vandross, MD',...",Academic medical center news release,"['Christiana Care Health System', 'CRISPR', 'l...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00001,Christiana Care Health System,This news release reports on preliminary resea...,Chemotherapy is a mainstay of treatment for ad...,https://www.healthnewsreview.org/wp-content/up...,Not Satisfactory,The news release mentioned this as a potential...,Satisfactory,The news release makes it clear that this is n...,Satisfactory,The news release mentions “many efforts under ...,Not Satisfactory,The news release doesn’t talk about specific h...,Not Satisfactory,In the second paragraph the news release expla...,Not Satisfactory,The news release uses cautious language. Howev...,Satisfactory,There’s no disease-mongering. The release says...,Not Satisfactory,The release is unable to claim a benefit in hu...,Satisfactory,The funders (NIH and State of Delaware) are no...,Satisfactory,The lead researcher calls the study “an exciti...
2,2,https://web.arch

In [ ]:
base_news_releases

,Unnamed: 0,url,text,images,top_img,keywords,authors,canonical_link,title_1,meta_data,movies,publish_date,source,summary,id,link,title_2,description,original_title,rating,reviewers,category,tags,source_link,news_id,news_source,Our Review Summary,Why This Matters,img,question_0_answer,question_0_explanation,question_1_answer,question_1_explanation,question_2_answer,question_2_explanation,question_3_answer,question_3_explanation,question_4_answer,question_4_explanation,question_5_answer,question_5_explanation,question_6_answer,question_6_explanation,question_7_answer,question_7_explanation,question_8_answer,question_8_explanation,question_9_answer,question_9_explanation,length
0,0,https://web.archive.org/web/20181218015531/htt...,"MADISON, Wis. -- More than 700 million adults ...",['https://web.archive.org/web/20181218015531im...,https://web.archive.org/web/20181218015531im_/...,[],[],https://web.archive.org/web/20181218015531/htt...,Tiny implantable device short-circuits hunger ...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00000,https://www.healthnewsreview.org/news-release-...,Implantable device ‘aids weight loss’? Let’s w...,This news release speculates that a study in r...,Tiny implantable device short-circuits hunger ...,2,"['Andrew Holtz, MPH', 'Yoni Freedhoff, MD', 'K...",University news release,"['implant', 'University of Wisconsin-Madison',...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00000,University of Wisconsin-Madison,UW-Madison graduate student Guang Yao (left) a...,The history of medical research is littered wi...,NaN,Not Satisfactory,Although this device is still in early testing...,Satisfactory,The release states that the device is experime...,Satisfactory,The release does report that there is an exist...,Not Satisfactory,"The release does not discuss potential harms, ...",Satisfactory,We will give the release a satisfactory rating...,Not Satisfactory,"By definition, researchers do not know whether...",Not Satisfactory,The lead sentence of the release proclaims tha...,Not Satisfactory,The news release refers to “several advantages...,Satisfactory,The release states that funding was provided b...,Not Satisfactory,The release says this device could be an impro...,3633.0
1,1,https://web.archive.org/web/20181217203805/htt...,"Wilmington, DE, December 17, 2018 - The CRISPR...",['https://web.archive.org/web/20181217203805im...,https://web.archive.org/web/20181217203805im_/...,[],[],https://web.archive.org/web/20181217203805/htt...,Scientists report CRISPR restores effectivenes...,"{'viewport': 'width=device-width, initial-scal...",[],2018-12-29 02:00:00,https://web.archive.org,NaN,news_reviews_00001,https://www.healthnewsreview.org/news-release-...,Summary of CRISPR lung cancer research downpla...,This release needed a strong caution that obse...,Scientists report CRISPR restores effectivenes...,3,"['Mary Chris Jaklevic', 'Andrae Vandross, MD',...",Academic medical center news release,"['Christiana Care Health System', 'CRISPR', 'l...",https://www.eurekalert.org/pub_releases/2018-1...,news_reviews_00001,Christiana Care Health System,This news release reports on preliminary resea...,Chemotherapy is a mainstay of treatment for ad...,https://www.healthnewsreview.org/wp-content/up...,Not Satisfactory,The news release mentioned this as a potential...,Satisfactory,The news release makes it clear that this is n...,Satisfactory,The news release mentions “many efforts under ...,Not Satisfactory,The news release doesn’t talk about specific h...,Not Satisfactory,In the second paragraph the news release expla...,Not Satisfactory,The news release uses cautious language. Howev...,Satisfactory,There’s no disease-mongering. The release says...,Not Satisfactory,The release is unable to claim a benefit in hu...,Satisfactory,The funders (NIH and State of Delaware) are no...,Satisfactory,The lead researcher calls the study “an exciti...,7042.0